In [ ]:
from DefinitionMethod_s import unifier_dataframe,attribuer_ids_df_hash_simple,lancer_scrapping_med,lancer_scrapping_nabady,lancer_scrapping_dabadoc,nettoyer_dataframe_medecins,detecter_changement_par_plateforme,lancer_scrapping_docdialy
import pandas as pd 
from pymongo import MongoClient
from insertion import inserer_dataframe
from ConnexionMongo_DB import get_db
from datetime import datetime
date_extraction=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
df1=lancer_scrapping_dabadoc()
df1["Date Extraction"]=date_extraction
#df1
df2=lancer_scrapping_med()
df2["Date Extraction"]=date_extraction
#df2
df3=lancer_scrapping_nabady()
df3["Date Extraction"]=date_extraction
#df3
#df4=lancer_scrapping_docdialy()
df4=pd.read_excel(r"C:\Users\HP\Desktop\Nouveau dossier (2)\Fichier_DocDialy_ma.xlsx")
df4=nettoyer_dataframe_medecins(df4,"Docdialy")
df4["Date Extraction"]=date_extraction
#df4
#Unifier le modele :
colonnes_standard=["ID Medecin","Statut","Prénom","Nom","Sexe","Spécialité","Ville","Date de Naissance","Téléphone","WhatsApp","Fix","Email","Duree","Date de Connexion","Date Extraction","Plateforme"]
#df1=unifier_dataframe(df1,"Dabadoc")
#df2=unifier_dataframe(df2,"Med")
#df3=unifier_dataframe(df3,"Nabady")
#df4=unifier_dataframe(df4,"Docdialy")
df_final=pd.concat([df1,df2,df3,df4],ignore_index=True)
#Associer des identifiants à la spécialité:
db = get_db()
df_final, id_maps = attribuer_ids_df_hash_simple(df_final, db, collection_name="collection_globale")
#df_final.to_excel(r"C:\Users\HP\Desktop\Data_Folder\df_Version.xlsx",index=False)
####Enregistrer dans MongoDB:
inserer_dataframe(df_final,get_db(),"collection_globale")
####Detecter les elements présents,absents et modifiés:
detecter_changement_par_plateforme()


succes
succes
20334 documents insérés dans 'collection_globale'
Comparaison entre 2025-08-06 15:33:23 et 2025-08-10 14:59:56
Traitement de la plateforme: {'id': 'plt_4eb46c2ac0', 'libelle': 'Nabady'}
Plateforme: {'id': 'plt_4eb46c2ac0', 'libelle': 'Nabady'} -Ajoutes: 3 -Supprimes : 0 -Modifies : 127
Traitement de la plateforme: {'id': 'plt_6ef98737ab', 'libelle': 'Med'}
Plateforme: {'id': 'plt_6ef98737ab', 'libelle': 'Med'} -Ajoutes: 2 -Supprimes : 3 -Modifies : 1
Traitement de la plateforme: {'id': 'plt_8dcb4740ba', 'libelle': 'Docdialy'}
Plateforme: {'id': 'plt_8dcb4740ba', 'libelle': 'Docdialy'} -Ajoutes: 0 -Supprimes : 0 -Modifies : 0
Traitement de la plateforme: {'id': 'plt_b631518c42', 'libelle': 'Dabadoc'}
Plateforme: {'id': 'plt_b631518c42', 'libelle': 'Dabadoc'} -Ajoutes: 1 -Supprimes : 0 -Modifies : 5
Félicitations la comparaison est terminée entre les deux versions!!!!


,Plateforme


In [ ]:
####Un script des différents types d'automatisation manuelle:
from DefinitionMethod import lancer_scrapping_med,lancer_scrapping_nabady ,nettoyer_dataframe_medecins,detecter_changement_par_plateforme,lancer_scrapping_dabadoc,lancer_scrapping_docdialy
import pandas as pd 
from pymongo import MongoClient
from insertion import inserer_dataframe
from ConnexionMongo_DB import get_db
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
date_extraction=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
def pipeline(mode="Séquentiel"):
    print("Mode Séquentiel")
    if mode=="Séquentiel":
        #Le bloc de scrapping:
        df1=lancer_scrapping_dabadoc()
        df1["Date Extraction"]=date_extraction
        df2=lancer_scrapping_med()
        df2["Date Extraction"]=date_extraction
        df3=lancer_scrapping_nabady()
        df3["Date Extraction"]=date_extraction
        #df4=lancer_scrapping_docdialy()
        df4=pd.read_excel(r"C:\Users\HP\Desktop\Nouveau dossier (2)\Fichier_DocDialy_ma.xlsx")
        df4=nettoyer_dataframe_medecins(df4,"Docdialy")
        df4["Date Extraction"]=date_extraction
        #Unifier le modele :
        df1=unifier_dataframe(df1,"Nabady")
        df2=unifier_dataframe(df2,"Med")
        df3=unifier_dataframe(df3,"Dabadoc")
        df4=unifier_dataframe(df4,"Docdialy")
        df_final=pd.concat([df1,df2,df3,df4],ignore_index=True)
        #Ajouter des identifiants:
        df_final=ajouter_id(df_final)
        df_final.to_excel(r"C:\Users\HP\Desktop\Data_Folder\df_Version.xlsx",index=False)
        ####Enregistrer dans MongoDB:
        inserer_dataframe(df_final,get_db(),"collection_globale")
        ####Detecter les elements présents,absents et modifiés:
        detecter_changement_par_plateforme()
    elif mode=="Parallèle":
        print("Mode Parallèle")
        #Lancer le script de scripping en parallèle pour toutes les plateformes
        with ThreadPoolExecutor() as executor:
            futures={
                "f1":executor.submit(lancer_scrapping_dabadoc),
                "f2":executor.submit(lancer_scrapping_med),
                "f3":executor.submit(lancer_scrapping_nabady),
                "f4":executor.submit(lancer_scrapping_docdialy)
            }
            df1=futures["f1"].result()
            df2=futures["f2"].result()
            df3=futures["f3"].result()
            df4=futures["f4"].result()
            for df in [df1,df2,df3,df4]:
                df["Date Extraction"]=date_extraction
            #Lancer le shéma pour toutes les plateformes
            with ThreadPoolExecutor as executor:
                futures={
                    "u1":executor.submit(unifier_dataframe,df1,"Dabadoc"),
                    "u2":executor.submit(unifier_dataframe,df2,"Med"),
                    "u3":executor.submit(unifier_dataframe,df3,"Nabady"),
                    "u4":executor.submit(unifier_dataframe,df4,"Docdialy")
                }
                df1=futures["u1"].result()
                df2=futures["u2"].result()
                df3=futures["u3"].result()
                df4=futures["u4"].result()
            df_final=pd.concat([df1,df2,df3,df4],ignore_index=True)
             #Ajouter des identifiants:
            df_final=ajouter_id(df_final)
            df_final.to_excel(r"C:\Users\HP\Desktop\Data_Folder\df_Version.xlsx",index=False)
            ####Enregistrer dans MongoDB:
            inserer_dataframe(df_final,get_db(),"collection_globale")
            ####Detecter les elements présents,absents et modifiés:
            detecter_changement_par_plateforme()
    else:
        with ThreadPoolExecutor as executor:
            features={
                "df1":executor.submit(lancer_scrapping_dabadoc),
                "df2":executor.submit(lancer_scrapping_med),
                "df3":executor.submit(lancer_scrapping_nabady),
                "df4":executor.submit(lancer_scrapping_docdialy)
            }   
        df1=features["df1"].result()
        df2=features["df2"].result()
        df3=features["df3"].result()
        df4= features["df4"].result()
        for df in ["df1","df2","df3","df4"]:
            df["Date Extraction"]=date_extraction 
        #Unifier le modele :
        df1=unifier_dataframe(df1,"Nabady")
        df2=unifier_dataframe(df2,"Med")
        df3=unifier_dataframe(df3,"Dabadoc")
        df4=unifier_dataframe(df4,"Docdialy")
        df_final=pd.concat([df1,df2,df3,df4],ignore_index=True)
        #Ajouter des identifiants:
        df_final=ajouter_id(df_final)
        df_final.to_excel(r"C:\Users\HP\Desktop\Data_Folder\df_Version.xlsx",index=False)
        ####Enregistrer dans MongoDB:
        inserer_dataframe(df_final,get_db(),"collection_globale")
        ####Detecter les elements présents,absents et modifiés:
        detecter_changement_par_plateforme()    
        